In [4]:

#maintainDict.py




def connectDB(whichCollection):
    #connect to MongoDB
    client = MongoClient('localhost', 27017)
    db = client.dpdb
    if whichCollection == 'dpdict':
        print '---Connect to collection: ', whichCollection
        collectionName = db.dpdict
    elif whichCollection == 'geweiDict':
        print '---Connect to collection: ', whichCollection
        collectionName = db.geweiDict
    else
        print '---wrong collection!'
    
    return collectionName


def dumpDict():
    """ dump from collection 'dpdict' of dpdb
    """
    
    dpdict = connectDB('dpdict')
    # http://api.mongodb.org/python/current/api/pymongo/cursor.html
    
    #dump 84 French words
    #results = dpdict.find({'fr': 'y'},{'_id':0}).sort('hw')
    
    #dump other 39111 words except French words
    results = dpdict.find({'fr': {'$exists': None}},{'_id':0})
    
    cnt = 1
    for result in results:
        word = result.get('hw')
        print cnt, ': ', word
        fname = dictpath + word + '_' + str(cnt) + '.json'
        if isfile(fname): print '---has existed---', fname
        fdict = open(fname, 'w')
        #fdict = open(dictpath + str(cnt) + '.' + word + '.json', 'w')
        fdict.write(dumps(result, sort_keys=True, indent=4,separators=(',', ': ')))
        fdict.close()
        cnt += 1        
        #if cnt > 10: break




def loadOneFile(fname):
    """ load one json file into collection 'geweiDict' of dpdb
    """
    
    gwdict = connectDB('geweiDict')
    
    #print fname
    with open(fname, 'r') as fentry:
        raw_doc = fentry.read()    
    #print raw_doc, '\n', type(raw_doc)
    doc_id = gwdict.insert(json.loads(raw_doc))
    print doc_id

    
def loadDict(pfname):
    """ load one json file or all json files in a directory into collection 'geweiDict' of dpdb
        pfname can be name of a file or a path
    """
    
    if isdir(pfname):
        cnt = 1
        files = [ f for f in listdir(pfname) if isfile(join(pfname,f)) ]
        for file in files:
            print cnt, '---', file
            loadOneFile(pfname + file)
            cnt += 1
    elif isfile(pfname):
        print pfname
        loadOneFile(pfname)
    else:
        print '---check ', pfname
    





#way.g.wang@outlook.com
ref5 = 'collegiate'
key5 = '3976dd43-e75c-4ffc-ac79-893c46887efb'

def grab_xml(word, ref, key):
    #response = urllib2.urlopen('http://www.dictionaryapi.com/api/v1/references/learners/xml/decipher?
    #                             key=8e5785eb-6777-4d84-8684-4e92a96733d6')
    req = 'http://www.dictionaryapi.com/api/v1/references/' + ref + '/xml/' + urllib2.quote(word) + '?key=' + urllib2.quote(key)
    print req
    response = urllib2.urlopen(req)
    return response.read()


def pullDictFromMW(filepath, wordlist):
    fwords = open(filepath + wordlist, 'r')
    #fwordNoXML = open(filepath + 'fwordNoXML.txt', 'w')
    
    cnt = 0
    for line in fwords:       
        jsonBody = {}
        
        cnt += 1
        #if cnt > 1: break 
            
        word = line.strip()
        print word, '---------\n'

        html = grab_xml(word, ref5, key5)
        print html, '\n'
        
        i = 0
        try:
            root = ET.fromstring(html)
                
            ewcnt = 0
            for entry in root.iter('entry'):                    
                eword = entry.find('ew').text               
                if eword != word: continue
                    
                ewcnt += 1
                print eword, ': ', ewcnt
                if ewcnt == 1:
                    fj = open(filepath + word + '.json', 'w')
                elif ewcnt > 1:
                    fj = open(filepath + word + '_' + str(ewcnt) + '.json', 'w')
                

                addhis = []
                addhis.append(time.strftime("%Y%m%d"))
                addhis.append('MWCollegiate') #pull from M-W Collegiate API
                jsonBody['addhis'] = addhis
                jsonBody['hw'] = eword
                
                pron = entry.find('pr')
                if pron != None: 
                    print pron.text                
                    jsonBody['prn'] = pron.text

                pos = entry.find('fl')
                if pos != None: 
                    print pos.text                
                    jsonBody['pos'] = pos.text
                    
                defis = []
                for defi in entry.iter('dt'):
                    #print defi.text
                    for itext in defi.itertext():
                        print '---itertext:', itext
                        if len(itext) > 1: defis.append(itext)
                if defis: jsonBody['sublist'] = defis                
                
                infls = []
                for infl in entry.iter('ure'):
                    print '===ure:', infl.text
                    infls.append(infl.text.replace('*', ''))
                for infl in entry.iter('if'):
                    print '===if:', infl.text
                    infls.append(infl.text.replace('*', ''))
                if infls: jsonBody['infl'] = infls

       
                for sound in entry.iter('wav'):
                    i += 1
                    #http://media.merriam-webster.com/soundc11/h/heart001.wav
                
                    if word[0].isdigit(): subdir = 'number'
                    else: subdir = word[0]
                
                    wavurl = 'http://media.merriam-webster.com/soundc11/' + subdir + '/' + sound.text
                    #print wavurl
            
                    if i==1:
                        #print word + ': ' + sound.text
                        #pass
                        urllib.urlretrieve(wavurl, filepath + word + '.wav')
                    else:
                        #print sound.text
                        pass
                        ##urllib.urlretrieve(wavurl, filepath + 'subentry\\' + sound.text)
                        
                if jsonBody:
                    print jsonBody
                    json.dump(jsonBody, fj, sort_keys=True, indent=4, separators=(',', ': '))
            
                fj.close()                        
                    
        except ParseError:
            print '---in ParseError'
            continue
        else:
            print '---in else'
            continue
            
        
        #if i==0: fwordNoXMLorWav.write(line)     

    fwords.close()
    #fwordNoXMLorWav.close()


#dumpDict()

#loadDict(updatepath)
#loadDict(updatepath + 'meager_26608.json')

#loadDict(addpath)
#loadDict(addpath + 'vantage.json')
        
#pullDictFromMW(rootpath + 'ulysses\\', 'aboutUlysses_tidy_notIndict.txt')



heaventree ---------

http://www.dictionaryapi.com/api/v1/references/collegiate/xml/heaventree?key=3976dd43-e75c-4ffc-ac79-893c46887efb
<?xml version="1.0" encoding="utf-8" ?>
<entry_list version="1.0">
	<suggestion>Aventura</suggestion>
	<suggestion>venture</suggestion>
	<suggestion>aventurine</suggestion>
	<suggestion>Aventine</suggestion>
	<suggestion>heaven-sent</suggestion>
</entry_list> 

---in else
indecorous ---------

http://www.dictionaryapi.com/api/v1/references/collegiate/xml/indecorous?key=3976dd43-e75c-4ffc-ac79-893c46887efb
<?xml version="1.0" encoding="utf-8" ?>
<entry_list version="1.0">
	<entry id="indecorous"><ew>indecorous</ew><hw>in*de*co*rous</hw><sound><wav>indeco02.wav</wav><wpr>(+)in-!de-k(u-)rus</wpr></sound><pr>(ˌ)in-ˈde-k(ə-)rəs; ˌin-di-ˈkȯr-əs</pr><fl>adjective</fl><et>Latin <it>indecorus,</it> from <it>in-</it> + <it>decorus</it> decorous</et><def><date>1668</date><dt>:not decorous :conflicting with accepted standards of good conduct or good taste</dt></de

In [54]:
import string
a = 's*a*y'
aa = string.split('s*a*y','*')
print a
aa = a.replace('*', '')
print aa




s*a*y
say
